<a href="https://www.kaggle.com/code/sabienn/xgb-hotel-reservation-predictions-prev-logreg?scriptVersionId=118849024" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Hotel Cancellation prediction
<br>blank
<br>V1: Created Logistic regression model that predicts hotel reservation cancellations 
<br>V2: Added categorical data using pandas function and ran another model
<br>V3: Dropped Booking ID from categorical data because it should not impact anything, plus it decreased the runtime of the notebook. 
<br>V4: Added f1 scoring. Interesting observation being that the model returns a higher score when 'Not_Canceled' is considered the positive label. Interested in seeing other scoring methods, f1 scoring potentially might not fit the situation. Another interesting observation, adding categorical data more significantly increased the f1 score when positive label was 'Canceled'.
<br>V5: Ensembling done by using a voting classifier. Considering the performance of the past version, our ensembled models have provided a better performance. However, the increase of performance is very marginal at this point. If anyone wonders why I am using logisitc regression, I wanted to analyze the performance of a linear model on this data. Maybe in future notebooks or Versions I will use a non linear model on this same data. 
<br>V6: Switched model to XGBoosting, and saw significant improvement. It is almost laughable how much better the performance of XGBoosting. Moreover, I added some preprocessing things, but I don't think there are any changes that came out of it. I believe a less "easy" to work with dataset would have had improvements
<br>V7: oops fixed a small error
<br>V8: hyperparameter tuning allowed for slight improvement in performance

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hotel-reservations-classification-dataset/Hotel Reservations.csv


In [2]:
hotel_filepath = "/kaggle/input/hotel-reservations-classification-dataset/Hotel Reservations.csv"

hotel_data = pd.read_csv(hotel_filepath)

In [3]:
hotel_data.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


check for missing values (for practice)

In [4]:
# Determine the percentage of missing data for each feature
missing_data = hotel_data.isnull().mean()

# Print the percentage of missing data for each feature
print(missing_data)

Booking_ID                              0.0
no_of_adults                            0.0
no_of_children                          0.0
no_of_weekend_nights                    0.0
no_of_week_nights                       0.0
type_of_meal_plan                       0.0
required_car_parking_space              0.0
room_type_reserved                      0.0
lead_time                               0.0
arrival_year                            0.0
arrival_month                           0.0
arrival_date                            0.0
market_segment_type                     0.0
repeated_guest                          0.0
no_of_previous_cancellations            0.0
no_of_previous_bookings_not_canceled    0.0
avg_price_per_room                      0.0
no_of_special_requests                  0.0
booking_status                          0.0
dtype: float64


The objective of this is predict whether or not the customer will cancel the reservation or not. The first way I will attempt to implement logistic regression. For simplicity, I will only use numerical data. 

In [5]:
categorical_columns = hotel_data.select_dtypes(include=["object"]).columns
X = hotel_data.drop(categorical_columns, axis=1)
y = hotel_data["booking_status"]

# initialize StandardScaler
scaler = StandardScaler()

# fit and transform the data
scaled_data = scaler.fit_transform(X)

# create a LabelEncoder object
le = LabelEncoder()

# fit and transform the categorical data into numerical data
y = le.fit_transform(y)

#creating training and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

Implementing ensembling model and fitting

In [6]:
#model_1 = LogisticRegression(max_iter = 5000, random_state=0, C=1)
#model_3 = LogisticRegression(max_iter = 5000, random_state=2, C=10, penalty='l2')
model_3 = xgb.XGBClassifier(max_depth=6, max_leaves=2, n_jobs=0, learning_rate=0.4, min_child_weight=3);
#final_model = VotingClassifier(
    #estimators=[('lr', model_1), ('lr_2', model_2), ('xgb', model_3)], voting='hard')
#final_model.fit(X_train, y_train)
model_3.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.4, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=2, min_child_weight=3,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [7]:
from sklearn.metrics import f1_score
predictions = model_3.predict(X_test)

score = f1_score(y_test, predictions)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: {}%".format((accuracy*100).round(2)))

print("F-1 Score: ", score)

Accuracy: 87.74%
F-1 Score:  0.9111679181977952


Accuracy for a logistic regression model is approximately 77%. Accuracy for XGBoosting is 87%, about 10% higher. after tuning max_depth we got a 2% increase for accuracy. changing learning rate caused slight improvement. 

In [8]:
hotel_data = pd.read_csv(hotel_filepath)
df_no_status = hotel_data.drop("booking_status", axis=1)
df_no_status = df_no_status.drop('Booking_ID', axis=1)

categorical_columns = df_no_status.select_dtypes(include=["object"]).columns
encoded_hotel_data = pd.get_dummies(df_no_status, columns=categorical_columns)

X = encoded_hotel_data
y = hotel_data["booking_status"]

# initialize StandardScaler
scaler = StandardScaler()

# fit and transform the data
scaled_data = scaler.fit_transform(X)

# create a LabelEncoder object
le = LabelEncoder()

# fit and transform the categorical data into numerical data
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

now to train the model

In [9]:
model_3 = xgb.XGBClassifier(max_depth=6, max_leaves=2, n_jobs=0, learning_rate=0.4, min_child_weight=3);

model_3.fit(X_train, y_train)

predictions = model_3.predict(X_test)
score = f1_score(y_test, predictions)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: {}%".format((accuracy*100).round(2)))
print(" f1 score: ", score)


Accuracy: 89.61%
 f1 score:  0.9238075990299112


Categorical data slightly increases accuracy and score. 